 Importing libraries

In [92]:

import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

reading the Treebank tagged sentences

In [93]:
nltk.download("treebank")
wsj = list(nltk.corpus.treebank.tagged_sents())

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\alire\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


first few tagged sentences

In [94]:
print(wsj[:5])

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.')], [('A', 'DT'), ('f

Splitting into train and test

In [95]:
random.seed(1234)
train_set, test_set = train_test_split(wsj, test_size=0.3)

print(len(train_set))
print(len(test_set))
print(train_set[:5])

2739
1175
[[('Few', 'JJ'), ('people', 'NNS'), ('think', 'VBP'), ('0', '-NONE-'), ('Mr.', 'NNP'), ('Spiegel', 'NNP'), ('wants', 'VBZ'), ('*-1', '-NONE-'), ('to', 'TO'), ('run', 'VB'), ('a', 'DT'), ('bread-and-butter', 'JJ'), ('thrift', 'NN'), (',', ','), ('which', 'WDT'), ('current', 'JJ'), ('rules', 'NNS'), ('would', 'MD'), ('force', 'VB'), ('Columbia', 'NNP'), ('to', 'TO'), ('become', 'VB'), ('*T*-2', '-NONE-'), ('.', '.')], [('Revenue', 'NN'), ('declined', 'VBD'), ('8', 'CD'), ('%', 'NN'), ('to', 'TO'), ('$', '$'), ('85.7', 'CD'), ('million', 'CD'), ('*U*', '-NONE-'), (',', ','), ('from', 'IN'), ('$', '$'), ('93.3', 'CD'), ('million', 'CD'), ('*U*', '-NONE-'), ('a', 'DT'), ('year', 'NN'), ('earlier', 'JJR'), ('.', '.')], [('Mrs.', 'NNP'), ('Ward', 'NNP'), ('says', 'VBZ'), ('0', '-NONE-'), ('she', 'PRP'), ('often', 'RB'), ('defended', 'VBD'), ('her', 'PRP'), ('to', 'TO'), ('colleagues', 'NNS'), ('who', 'WP'), ('*T*-91', '-NONE-'), ('called', 'VBD'), ('her', 'PRP'), ('a', 'DT'), ('gran

 Getting list of tagged words

In [96]:
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

70485

tokens 

In [97]:
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Few', 'people', 'think', '0', 'Mr.', 'Spiegel', 'wants', '*-1', 'to', 'run']

vocabulary

In [98]:
V = set(tokens)
print(len(V))

10366


number of tags

In [99]:
T = set([pair[1] for pair in train_tagged_words])
len(T)

46

In [100]:
print(T)

{'-NONE-', 'VBG', 'RBS', 'DT', 'NNPS', 'CC', 'WP$', '#', 'VBN', 'MD', 'VBD', 'RB', ':', 'JJR', 'SYM', 'IN', 'EX', '.', '``', 'TO', 'WDT', '-RRB-', '$', 'RP', 'JJ', 'RBR', 'VBP', 'NNS', 'WP', 'VB', 'PDT', 'POS', 'FW', '-LRB-', 'CD', 'UH', 'NN', 'LS', 'PRP', ',', 'JJS', 'VBZ', "''", 'PRP$', 'WRB', 'NNP'}


compute word given tag: Emission Probabilit

In [101]:
def word_given_tag(word, tag, train_bag=train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]
    count_w_given_tag = len(w_given_tag_list)

    return (count_w_given_tag, count_tag)

compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

In [102]:
def t2_given_t1(t2, t1, train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t == t1])
    count_t2_t1 = 0
    for index in range(len(tags) - 1):
        if tags[index] == t1 and tags[index + 1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

creating t x t transition matrix of tags. 

each column is t2, each row is t1.

thus M(i, j) represents P(tj given ti)

In [103]:
tags_matrix = np.zeros((len(T), len(T)), dtype="float32")
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0] / t2_given_t1(t2, t1)[1]

convert the matrix to a df for better readability

In [104]:
tags_df = pd.DataFrame(tags_matrix, columns=list(T), index=list(T))

In [105]:
tags_df.head(10)

,-NONE-,VBG,RBS,DT,NNPS,CC,WP$,#,VBN,MD,...,NN,LS,PRP,",",JJS,VBZ,'',PRP$,WRB,NNP
-NONE-,0.072980,0.081668,0.000217,0.051694,0.000000,0.010860,0.0,0.0,0.011729,0.015421,...,0.021286,0.000000,0.052129,0.051043,0.000434,0.038228,0.000000,0.004561,0.000869,0.038445
VBG,0.075435,0.002901,0.000967,0.193424,0.000967,0.015474,0.0,0.0,0.016441,0.000000,...,0.146035,0.000000,0.012573,0.020309,0.000000,0.000967,0.001934,0.026112,0.000000,0.032882
RBS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DT,0.001933,0.010192,0.002460,0.001406,0.002987,0.000000,0.0,0.0,0.011070,0.001230,...,0.472676,0.000000,0.000351,0.001230,0.010016,0.004744,0.000176,0.000176,0.000000,0.125813
NNPS,0.005952,0.000000,0.000000,0.005952,0.005952,0.071429,0.0,0.0,0.000000,0.035714,...,0.011905,0.000000,0.005952,0.136905,0.000000,0.023810,0.000000,0.000000,0.000000,0.345238
CC,0.008202,0.025868,0.000000,0.105994,0.002524,0.000000,0.0,0.0,0.017035,0.011356,...,0.116088,0.000631,0.044795,0.009464,0.001893,0.022082,0.000000,0.014511,0.003785,0.156467
WP$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.444444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VBN,0.551560,0.012890,0.000678,0.056988,0.000000,0.009498,0.0,0.0,0.025102,0.000000,...,0.069878,0.000000,0.002714,0.005427,0.000000,0.000678,0.004071,0.012212,0.000000,0.011533
MD,0.009331,0.000000,0.000000,0.003110,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.001555,0.000000,0.001555,0.000000,0.000000,0.000000,0.000000,0.001555


### Viterbi Algorithm

In [106]:
len(train_tagged_words)

70485

Viterbi Heuristic

In [107]:
def Viterbi(words, train_bag=train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        # initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc[".", tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = (
                word_given_tag(words[key], tag)[0] / word_given_tag(words[key], tag)[1]
            )
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

### Evaluating on Test Set

Running the Viterbi algorithm on a few sample sentences since running it on the entire data set will take many hours

In [108]:
random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1, len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run[:1]

[[('Mr.', 'NNP'),
  ('Bernstein', 'NNP'),
  ('said', 'VBD'),
  ('0', '-NONE-'),
  ('he', 'PRP'),
  ('will', 'MD'),
  ('stay', 'VB'),
  ('until', 'IN'),
  ('Dec.', 'NNP'),
  ('31', 'CD'),
  ('and', 'CC'),
  ('work', 'VB'),
  ('with', 'IN'),
  ('his', 'PRP$'),
  ('successor', 'NN'),
  (',', ','),
  ('who', 'WP'),
  ('*T*-36', '-NONE-'),
  ('is', 'VBZ'),
  ('*-1', '-NONE-'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('named', 'VBN'),
  ('*-43', '-NONE-'),
  ('soon', 'RB'),
  ('.', '.')]]

tagging the test sentences

In [109]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end - start

In [110]:
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  99.33024287223816
[('Mr.', 'NNP'), ('Bernstein', 'NNP'), ('said', 'VBD'), ('0', '-NONE-'), ('he', 'PRP'), ('will', 'MD'), ('stay', 'VB'), ('until', 'IN'), ('Dec.', 'NNP'), ('31', 'CD'), ('and', 'CC'), ('work', 'NN'), ('with', 'IN'), ('his', 'PRP$'), ('successor', 'NN'), (',', ','), ('who', 'WP'), ('*T*-36', '-NONE-'), ('is', 'VBZ'), ('*-1', '-NONE-'), ('to', 'TO'), ('be', 'VB'), ('named', 'VBN'), ('*-43', '-NONE-'), ('soon', 'RB'), ('.', '.'), ('Like', 'IN'), ('Jaguar', 'NNP'), (',', ','), ('B.A.T', '-NONE-'), ('also', 'RB'), ('eased', 'VBD'), ('off', 'RP'), ('its', 'PRP$'), ('highs', '-NONE-'), ('in', 'IN'), ('afternoon', 'NN'), ('dealings', 'NNS'), ('.', '.'), ('The', 'DT'), ('Treasury', 'NNP'), ('said', 'VBD'), ('0', '-NONE-'), ('the', 'DT'), ('refunding', 'NN'), ('is', 'VBZ'), ('contingent', '-NONE-'), ('upon', 'IN'), ('congressional', 'JJ'), ('and', 'CC'), ('presidential', '-NONE-'), ('passage', 'NN'), ('of', 'IN'), ('an', 'DT'), ('increase', 'NN'), ('in', 

accuracy

In [111]:
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

In [112]:
accuracy = len(check) / len(tagged_seq)

In [113]:
accuracy

0.928

Testing

In [114]:
sentence_test = "Twitter is the best networking social site. Man is a social animal. Data science is an emerging field. Data science jobs are high in demand."
words = word_tokenize(sentence_test)

start = time.time()
tagged_seq = Viterbi(words)
end = time.time()
difference = end - start

In [115]:
print(tagged_seq)
print(difference)

[('Twitter', '-NONE-'), ('is', 'VBZ'), ('the', 'DT'), ('best', 'JJS'), ('networking', '-NONE-'), ('social', 'JJ'), ('site', '-NONE-'), ('.', '.'), ('Man', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('social', 'JJ'), ('animal', '-NONE-'), ('.', '.'), ('Data', 'NNP'), ('science', '-NONE-'), ('is', 'VBZ'), ('an', 'DT'), ('emerging', 'VBG'), ('field', 'NN'), ('.', '.'), ('Data', 'NNP'), ('science', '-NONE-'), ('jobs', 'NNS'), ('are', 'VBP'), ('high', 'JJ'), ('in', 'IN'), ('demand', 'NN'), ('.', '.')]
26.670320987701416
